## 使用训练和测试集评估XGBoost模型


我们可以用来评估算法性能的最简单方法就是使用训练集和测试集，可以把原有数据分成两份，一份用于训练，一份用于测试。


拆分大小取决于数据集的大小和细节，通常的选择就是使用0.33数据集进行测试。

下面我们继续使用下面使用 [Pima 印第安人糖尿病数据集](https://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes)开始提供完整的代码清单，假设它位于当前工作目录中（更新：[从这里下载](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)）。具有默认配置的 XGBoost 模型适合训练数据集并在测试数据集上进行评估。


In [42]:
# train-test split evaluation of xgboost model
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# load data
dataset =  loadtxt('pima-indians-diabetes.csv',delimiter=",",skiprows=1)
# split data into x and y
x = dataset[:,0:8]
y = dataset[:,8]
# split data into train and test sets
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=7)
# fit model on training data
model = XGBClassifier()
model.fit(x_train,y_train)
# make predictions for test data 
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]
# evaluate model
accuracy = accuracy_score(y_test,predictions)
print("Accuracy: %.2f%%" % (accuracy * 100))

Accuracy: 72.83%


以上是使用测试集和训练集评价模型，模型表现为72.83%，接下来我们使用K-Fold方法验证。

## K-Fold


交叉验证是一种用于机器学习算法表现评估方法，其方差小于单个训练测试集拆分，它是将数据分成k部分，每次数据分割称为fold，然后在k-1折叠上训练，然后在1折叠上测试，重复这一过程，使每个折叠都有可能成为测试集。


运行交叉验证后，可以使用均值和标准差来汇总，在给定测试集数据的情况下，对新数据算法表现进行更可靠的估计，因为他在不同的数据集上被训练过，因此更准确。


k的选择必须区分大小应足够大已成为问题的合理样本，同时允许对算法的训练评估的足够重复以提供对对看不见的数据的算法表现的公平估计。对于成千上万的观测值中，k值一般趣味5或10.


代码验证：

In [43]:
# k-fold cross validation evaluation for xgboost model
from numpy import loadtxt
import xgboost
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# load data
dataset = loadtxt('pima-indians-diabetes.csv',delimiter=",",skiprows=1)
# split data into x and y
x = dataset[:,0:8]
y = dataset[:,8]
# CV model
model = XGBClassifier()
kfold = KFold(n_splits=10)
results = cross_val_score(model,x,y,cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)"  % (results.mean()*100, results.std()*100))

Accuracy: 74.22% (5.75%)


如果对于多分类的预测性问题，或者类是不平衡的（一个类的实例并另一个类多得多），那么在执行交叉验证时创建分层折叠可能是个好主意。


这具有在执行交叉验证中评估每个折叠中强制执行与在整个训练数据集中相容的类的分布的效果。


下面是修改为使用分层交叉验证来评估xgboost模型的相同的实例。

In [44]:
# stratified k-fold cross validation evaluation of xgboost model
from numpy import loadtxt
import xgboost
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# load data
dataset = loadtxt('pima-indians-diabetes.csv',delimiter=",",skiprows=1)
# split data into x and y
x = dataset[:,0:8]
y = dataset[:,8]
# CV model
model = xgboost.XGBClassifier()
kfold = StratifiedKFold(n_splits=10)
results = cross_val_score(model,x,y,cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

Accuracy: 74.35% (4.21%)


## 要使用哪一个呢？

*  通常，k-fold交叉验证适用于评估机器学习算法在看不见的数据上的表现的金标准，其中k设置为3，5，10.
*  当存在大量类或者每个类的实例不平衡时，使用分层交叉验证强制执行类分发。
*  使用慢速算法时，使用训练/测试分割有利于提高速度，并在使用大型数据集时产生较低较差的表现估计。

# 最后
* 建议试验并找到一种快速解决问题的验证技术，并产生可用于制定决策的合理表现估算。